In [30]:
from abc import ABC, abstractmethod
import logging
import logging.config
from pathlib import Path
from typing import Dict, Any, Optional
import json

In [26]:
class Logger(ABC):
    @abstractmethod
    def debug(self):
        pass

    @abstractmethod
    def info(self):
        pass

    @abstractmethod
    def warning(self):
        pass

    @abstractmethod
    def error(self):
        pass

    @abstractmethod
    def exception(self):
        pass

In [27]:
class DefaultLogger(Logger):
    def __init__(self, logger_name: Optional[str] = None) -> None:
        if logger_name is None:
            self.logger = logging.getLogger(__name__)
        else:
            self.logger = logging.getLogger(logger_name)

    def debug(self, msg: str) -> None:
        self.logger.debug(msg)

    def info(self, msg: str) -> None:
        self.logger.info(msg)

    def warning(self, msg: str) -> None:
        self.logger.warning(msg)

    def error(self, msg: str) -> None:
        self.logger.error(msg)

    def exception(self, msg: str) -> None:
        self.logger.exception(msg)

In [28]:
class CustomLogger(Logger):
    def __init__(
        self,
        log_file_name: str,
        log_folder: Path,
        config: Dict[str, Any],
        logger_name: str,
    ) -> None:
        if not log_folder.exists():
            log_folder.mkdir()
        config["handlers"]["file_handler"]["filename"] = str(
            log_folder
            / config["handlers"]["file_handler"]["filename"].format(
                filename=log_file_name
            )
        )
        logging.config.dictConfig(config)
        self.logger = logging.getLogger(logger_name)

    def debug(self, msg: str) -> None:
        self.logger.debug(msg)

    def info(self, msg: str) -> None:
        self.logger.info(msg)

    def warning(self, msg: str) -> None:
        self.logger.warning(msg)

    def error(self, msg: str) -> None:
        self.logger.error(msg)

    def exception(self, msg: str) -> None:
        self.logger.exception(msg)

In [29]:
class FakeLogger(Logger):
    def __init__(self, logger_name: Optional[str] = None) -> None:
        if logger_name is None:
            self.logger = logging.getLogger(__name__)
        else:
            self.logger = logging.getLogger(logger_name)

    def debug(self, msg: str) -> None:
        print(f"Fake logger debug method is called with {repr(msg)}")

    def info(self, msg: str) -> None:
        print(f"Fake logger info method is called with {repr(msg)}")

    def warning(self, msg: str) -> None:
        print(f"Fake logger warning method is called with {repr(msg)}")

    def error(self, msg: str) -> None:
        print(f"Fake logger error method is called with {repr(msg)}")

    def exception(self, msg: str) -> None:
        print(f"Fake logger exception method is called with {repr(msg)}")

In [14]:
logger = FakeLogger()

In [15]:
logger.debug("Hello this is debug message")

Fake logger debug method is called with 'Hello this is debug message'


In [16]:
logger.info("Hello this is info message")

Fake logger info method is called with 'Hello this is info message'


In [17]:
logger.error("Hello this is error message")

Fake logger error method is called with 'Hello this is error message'


In [18]:
try:
    1/0
except Exception as e:
    logger.exception(f"Error -> {repr(e)}")

Fake logger exception method is called with "Error -> ZeroDivisionError('division by zero')"


In [19]:
with open("logging.json", "r") as f:
    logging_configs = json.load(f)
logging_configs

{'version': 1,
 'disable_existing_loggers': False,
 'formatters': {'simple': {'format': '%(asctime)s - %(name)s - %(levelname)s : %(message)s'}},
 'handlers': {'console': {'class': 'logging.StreamHandler',
   'level': 'DEBUG',
   'formatter': 'simple',
   'stream': 'ext://sys.stdout'},
  'file_handler': {'class': 'logging.handlers.RotatingFileHandler',
   'level': 'DEBUG',
   'formatter': 'simple',
   'filename': '{filename}.log',
   'maxBytes': 209715200,
   'backupCount': 10,
   'encoding': 'utf8'}},
 'loggers': {'__main__': {'level': 'DEBUG',
   'handlers': ['console', 'file_handler'],
   'propagate': False}},
 'root': {'level': 'INFO', 'handlers': ['console']}}

In [20]:
logger = CustomLogger("mylog", Path(), logging_configs, __name__)

In [21]:
logger.debug("Hello this is debug message")

2020-06-27 21:30:51,899 - __main__ - DEBUG : Hello this is debug message


In [22]:
logger.info("Hello this is info message")

2020-06-27 21:30:53,658 - __main__ - INFO : Hello this is info message


In [23]:
logger.error("Hello this is error message")

2020-06-27 21:30:54,666 - __main__ - ERROR : Hello this is error message


In [24]:
try:
    1/0
except Exception as e:
    logger.exception(f"Error -> {repr(e)}")

2020-06-27 21:30:55,789 - __main__ - ERROR : Error -> ZeroDivisionError('division by zero')
Traceback (most recent call last):
  File "<ipython-input-24-f4cae4c95746>", line 2, in <module>
    1/0
ZeroDivisionError: division by zero
